In [1]:
from binance.client import Client
import pandas as pd
import time
import numpy as np
import os
from datetime import datetime
import random
import logging

# Set up logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("BinanceStrategy")

# Testnet credentials
API_KEY = "4ed11adeb60c67e697531b28b4dbea064bcfb105b11c06885e15d4e8d45c17ee"
API_SECRET = "d4012e3c49b1d718c2432f137efe32e29a842787e893b8c17d091689487cc274"

# Initialize Futures Testnet client
client = Client(
    api_key=API_KEY,
    api_secret=API_SECRET,
    testnet=True,
    # futures=True
)

# Utility: Safe API call with retry
def safe_api_call(func, max_retries=5, delay=5):
    retries = 0
    while retries < max_retries:
        try:
            return func()
        except Exception as e:
            logger.error(f"Error: {e}")
            if "Timestamp for this request is outside of the recvWindow" in str(e):
                logger.warning("Possible clock drift. Sync system time.")
            retries += 1
            sleep_time = delay * (2 ** retries) + random.uniform(0, 1)
            logger.info(f"Retrying in {sleep_time:.2f} seconds...")
            time.sleep(sleep_time)
    raise Exception("Max retries reached. Aborting.")

def stream_1m_data(symbol):
    symbol = symbol.upper()
    logger.info(f"Starting 1m stream from Binance Futures Testnet for {symbol}...")

    cols = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    df_live = pd.DataFrame(columns=cols)
    last_ts = None

    while True:
        def fetch_klines():
            return client.futures_klines(symbol=symbol, interval=Client.KLINE_INTERVAL_1MINUTE, limit=2)

        try:
            klines = safe_api_call(fetch_klines)
            if not klines:
                logger.warning(f"[{datetime.now()}] No data returned.")
                time.sleep(10)
                continue

            for kline in klines:
                ts = pd.to_datetime(kline[0], unit='ms')
                if last_ts is None or ts > last_ts:
                    new_row = {
                        'timestamp': ts,
                        'open': float(kline[1]),
                        'high': float(kline[2]),
                        'low': float(kline[3]),
                        'close': float(kline[4]),
                        'volume': float(kline[5])
                    }
                    df_live = pd.concat([df_live, pd.DataFrame([new_row])], ignore_index=True)
                    last_ts = ts
                    yield df_live.copy()

        except Exception as e:
            logger.error(f"[{datetime.now()}] Error: {e}")
            time.sleep(10)

        time.sleep(60)

In [2]:
def calculate_indicators(df):

    df = df.copy()

    df['sma_5'] = df['close'].rolling(window=5).mean()

    df['sma_10'] = df['close'].rolling(window=10).mean()

    df['sma_cross'] = np.where((df['sma_5'].shift(1) <= df['sma_10'].shift(1)) & (df['sma_5'] > df['sma_10']), "True", "False")

    return df                                                                

In [3]:
# position = 0  # Global variable to track the current position in the DataFrame

# trades = []
# active_trades = []
# exited_trades = []
# sl = 0.075

# tp = 0.03

# def strategy_implementation(df, symbol):
#     global position, active_trades, trades, exited_trades, sl, tp

#     df1 = calculate_indicators(df)  # Calculate indicators
#     df1.to_csv(f"{symbol}_live_data.csv", index=False)  # Save processed data

#     for i in range(position, len(df1)):
#         row = df1.iloc[i]
#         position = i + 1

#         print(f"Processing bar {i+1}/{len(df1)}: {row['timestamp']}")

#         # --- Exit logic ---
        
#         for trade in active_trades:
#                 if row['high'] >= trade['sl']:
#                     trade.update({
#                         'exit_time': row['timestamp'],
#                         'exit_reason': 'SL hit',
#                         'exit_price': trade['sl'],
#                         'pnl': trade['entry_price'] - trade['sl']
#                     })
#                     trades.append(trade)
#                     exited_trades.append(trade)
#                     print(f"Exiting trade at SL {trade['sl']} on {row['timestamp']}")

#                 elif row['low'] <= trade['tp']:
#                     trade.update({
#                         'exit_time': row['timestamp'],
#                         'exit_reason': 'TP hit',
#                         'exit_price': trade['tp'],
#                         'pnl': trade['entry_price'] - trade['tp']
#                     })
#                     trades.append(trade)
#                     exited_trades.append(trade)
#                     print(f"Exiting trade at TP {trade['tp']} on {row['timestamp']}")

#         active_trades = [t for t in active_trades if t not in exited_trades]


#         # --- Entry Logic ---
#         if (row['sma_cross'] == "True"):
#             entry_price = row['close']
#             active_trades.append({
#                 'entry_time': row['timestamp'],
#                 'position': 'short',
#                 'entry_price': entry_price,
#                 'sl': entry_price * (1 + sl),
#                 'tp': entry_price * (1 - tp)
#             })
#             print(f"Entering short position at {entry_price} with SL {active_trades[-1]['sl']} and TP {active_trades[-1]['tp']}")

#     # --- Metrics ---
#     df_trades = pd.DataFrame(trades)
#     if df_trades.empty:
#         return df_trades, {
#             'total_trades': 0, 'win_rate': 0, 'avg_pnl': 0,
#             'risk_reward_ratio': np.nan, 'max_drawdown': 0, 'avg_trade_per_day': 0
#         }

#     df_trades['wins'] = (df_trades['pnl'] > 0).astype(int)
#     df_trades['cumulative_pnl'] = df_trades['pnl'].cumsum()
#     initial_capital = df_trades['entry_price'].iloc[0]
#     df_trades['equity_curve'] = df_trades['cumulative_pnl'] + initial_capital
#     max_drawdown = (df_trades['equity_curve'].cummax() - df_trades['equity_curve']).max()
    
#     duration_days = (df['timestamp'].max() - df['timestamp'].min()).days + 1
#     metrics = {
#         'total_trades': len(df_trades),
#         'win_rate': df_trades['wins'].mean(),
#         'avg_pnl': df_trades['pnl'].mean(),
#         'risk_reward_ratio': (
#             df_trades[df_trades['pnl'] > 0]['pnl'].mean() /
#             abs(df_trades[df_trades['pnl'] < 0]['pnl'].mean())
#             if (df_trades['pnl'] < 0).any() else np.nan
#         ),
#         'max_drawdown': max_drawdown,
#         'avg_trade_per_day': len(df_trades) / duration_days
#     }

#     return df_trades, metrics

In [4]:
position = 0

active_trade = []
trade = []
exit_trade = []

sl = 500

tp = 120

def strategy_implementation(data, symbol):
    global position, active_trade, trade, exit_trade

    df = calculate_indicators(data)

    df.to_csv(f"{symbol}_live_data.csv", index=False)
    
    for i in range(position, len(df)):
        position = i + 1

        row = df.loc[i]

        # print(f"current_bar : {row['timestamp']}")

        if (row['sma_cross'] == "True") :

            print("We are ready to enter the market.")
            
            entry_price = row['close']

            sl_price = entry_price + sl
            tp_price = entry_price - tp

            active_trade.append({
                'entry_time': row['timestamp'],
                'position': 'short',
                'entry_price': entry_price,
                'sl': entry_price + sl,
                'tp': entry_price - tp
            })
            print(f"We enter the trade at : {row['timestamp']}, the entry price is : {entry_price}, SL : {sl_price} and TP : {tp_price}.")


        for trade_going in active_trade:

            # print(f"Active trade is : {trade_going}")

            if row['high'] >= trade_going['sl']:
                trade_going.update({
                    'exit_time': row['timestamp'],
                    'exit_reason': 'SL hit',
                    'exit_price': trade_going['sl'],
                    'pnl': trade_going['entry_price'] - trade_going['sl']
                })
                trade.append(trade_going)
                exit_trade.append(trade_going)
                print(f"Trade closed at {row['timestamp']} and the price is {row['high']}.")


            if row['low'] <= trade_going['tp']:
                trade_going.update({
                    'exit_time': row['timestamp'],
                    'exit_reason': 'TP hit',
                    'exit_price': trade_going['tp'],
                    'pnl': trade_going['entry_price'] - trade_going['tp']
                })
                trade.append(trade_going)
                exit_trade.append(trade_going)
                print(f"Trade close at {row['timestamp']} and the price is {row['low']}.")

        active_trade = [ trade for trade in active_trade if active_trade not in exit_trade]


    # --- Metrics ---
    df_trades = pd.DataFrame(trade)
    if df_trades.empty:
        return df_trades, {
            'total_trades': 0, 'win_rate': 0, 'avg_pnl': 0,
            'risk_reward_ratio': np.nan, 'max_drawdown': 0, 'avg_trade_per_day': 0
        }

    df_trades['wins'] = (df_trades['pnl'] > 0).astype(int)
    df_trades['cumulative_pnl'] = df_trades['pnl'].cumsum()
    initial_capital = df_trades['entry_price'].iloc[0]
    df_trades['equity_curve'] = df_trades['cumulative_pnl'] + initial_capital
    max_drawdown = (df_trades['equity_curve'].cummax() - df_trades['equity_curve']).max()
    
    duration_days = (df['timestamp'].max() - df['timestamp'].min()).days + 1
    metrics = {
        'total_trades': len(df_trades),
        'win_rate': df_trades['wins'].mean(),
        'avg_pnl': df_trades['pnl'].mean(),
        'risk_reward_ratio': (
            df_trades[df_trades['pnl'] > 0]['pnl'].mean() /
            abs(df_trades[df_trades['pnl'] < 0]['pnl'].mean())
            if (df_trades['pnl'] < 0).any() else np.nan
        ),
        'max_drawdown': max_drawdown,
        'avg_trade_per_day': len(df_trades) / duration_days
    }

    return df_trades, metrics

In [ ]:
def load_last_save_entry(symbol):
    filename = f"{symbol}_live_trades.csv"

    if os.path.exists(filename):
        try:
            df = pd.read_csv(filename)
            if not df.empty:
                return df.iloc[-1]['entry_time']
        except (pd.errors.EmptyDataError, KeyError):
            return None
    return None


if __name__ == "__main__":    
    
    symbol = input("Enter a symbol (e.g. BTCUSDT): ").strip().upper()
    df = stream_1m_data(symbol)

    Store = input("Do you want to store trades and metrics in CSV files? (yes/no): ").strip().lower()
    Store_data = True if Store in ['yes', 'y'] else False

    last_save_entry = load_last_save_entry(symbol)
    write_headers = not os.path.exists(f"{symbol}_live_trades.csv")

    for data in df :
        trade_df, metrics = strategy_implementation(data, symbol)

        if Store_data and not trade_df.empty:
            if last_save_entry is not None :
                new_trade = trade_df[trade_df['entry_time'] > last_save_entry]
            else :
                new_trade = trade_df

            if not new_trade.empty:
                new_trade.to_csv(
                    f"{symbol}_live_trades.csv",
                    mode = 'a',
                    header=write_headers,
                    index=False
                )
                write_headers=False
                last_save_entry = new_trade.iloc[-1]['entry_time']
                print(f"|{datetime.now()}| New trades saves to csv : {len(new_trade)}")
        

INFO:BinanceStrategy:Starting 1m stream from Binance Futures Testnet for BTCUSDT...
C:\Users\gauri\AppData\Local\Temp\ipykernel_35556\2341926827.py:72: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_live = pd.concat([df_live, pd.DataFrame([new_row])], ignore_index=True)


We are ready to enter the market.
We enter the trade at : 2025-06-29 16:56:00, the entry price is : 107400.4, SL : 107900.4 and TP : 107280.4.
We are ready to enter the market.
We enter the trade at : 2025-06-29 17:38:00, the entry price is : 107350.0, SL : 107850.0 and TP : 107230.0.
Trade close at 2025-06-29 17:39:00 and the price is 107260.0.
|2025-06-29 23:10:17.770919| New trades saves to csv : 1
Trade close at 2025-06-29 17:40:00 and the price is 107260.1.
Trade close at 2025-06-29 17:41:00 and the price is 107277.3.
Trade close at 2025-06-29 17:44:00 and the price is 107260.0.
We are ready to enter the market.
We enter the trade at : 2025-06-29 17:46:00, the entry price is : 107294.8, SL : 107794.8 and TP : 107174.8.
